In [1]:
# from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np

In [2]:
word_list = pd.read_csv("./불용어.csv" , encoding="utf-8-sig")

In [3]:
bul_list = []
for bul in word_list["불용어"] :
    bul_list.append(bul)

In [150]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
#         self.okt = Okt()
#         self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
#         ,"아", "휴", "아이구","다른" ,"얘기","아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]
        self.stopwords = bul_list
    def url2sentences(self, text):
#     article = Article(url, language='ko')
#     article.download()
#     article.parse()
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.kkma.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
                print(nouns)
        return nouns

In [122]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}


In [124]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    


In [123]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
            
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
            
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords
    

In [37]:
filename='./data/script/S000068.txt'

r = open(filename, mode='rt', encoding='utf-8-sig')
lines = r.readlines()
text0= ""
for line in lines:
    line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
    text0 += line + " "
    print(line)
f.close()

반갑습니다.여러분. 한 주 동안 공부 열심히 하셨습니까? 이제 또 새로운 주제로 여러분들을 공부해야 될 시간이 왔습니다. 아 이번 시간은 법적 영역에서 헌법 파트이지요. 헌법에는 어떤 내용들이 규정되어 있는지에 대해서 여러분들하고 같이 공부를 하게 됩니다.
그런데 여러분들이 헌법 파트 라고 하는 부분이 참 재미있고 참 심오합니다. 그런데 이제 한정된 시간 내에서 문제를 풀어야 되는 입장이지요. 이럴 때는 조금 여러분들의 노력이 여기서부터는 조금 더 많이 필요하지 않을까 라고 하는 생각이 들거든요.
어 여기 같은 경우에는 일단은 여러분들, 어 우리나라에서 어떻게 헌법에서 규정되어 있는지를 좀 습득 하셔야 되지 않을까 라고 하는 게 제 개인적인 생각이에요. 우리가 지금 헌법 파악이라고 하는 거 자체를 갖다가 수업하고 있는 내용이 아닌지라,
아 현실적으로 우리가 좀 성적이 좋은 성적을 받아서 여러분들이 원하는 바 빨리 이루는 것이 가장 중요하지 않겠습니까? 그렇다면 대한민국 헌법에 규정되어 있는 내용들이 무엇 인지를 조금 단순 암기,죄송합니다. 이런 말씀을 드려서,
단순 암기를 하셔야 될 부분들도 분명히 있습니다. 당연히 여러분들이 제도적인 측면이기 때문에 우리나라에서 어떤 제도를 지금 어떠한 방법으로 쓰고 있고,
어떠한 일 자, 어떠한 절차 이런 것들을 갖다가 하고 있는지는 이거는 뭐 이해의 대상이 된다 라기 보다는 아무래도 여러분들이 숙지의 대상이 되지 않을까 라고 하는 생각이 들어요. 그래서 이 헌법 파트에 있는 여러분들 기본권이 나와 있고 그리고 국회라고 하는 부분,
대통령과 행정부, 사법부 그리고 헌법재판소, 이런 내용들이 주로 이루어져 있습니다. 그러니까 우리나라에서는 이런 것들을 어떻게 규정하고 있는지를 좀 차근차근 정리하실 필요가 있지 않을까 싶습니다.
그러면 이 헌법 내용 중에서 이제 첫 번째로 우리가 같이 공부한 거에 될 내용은 우리나라 헌법은 구체적으로 어떤 원리 속에서 이루어 지고 있는지 즉,헌법의 기본 원리가 무엇인지에 대해서 말씀을 드려보도

NameError: name 'f' is not defined

In [38]:
text0

'반갑습니다.여러분. 한 주 동안 공부 열심히 하셨습니까? 이제 또 새로운 주제로 여러분들을 공부해야 될 시간이 왔습니다. 아 이번 시간은 법적 영역에서 헌법 파트이지요. 헌법에는 어떤 내용들이 규정되어 있는지에 대해서 여러분들하고 같이 공부를 하게 됩니다. 그런데 여러분들이 헌법 파트 라고 하는 부분이 참 재미있고 참 심오합니다. 그런데 이제 한정된 시간 내에서 문제를 풀어야 되는 입장이지요. 이럴 때는 조금 여러분들의 노력이 여기서부터는 조금 더 많이 필요하지 않을까 라고 하는 생각이 들거든요. 어 여기 같은 경우에는 일단은 여러분들, 어 우리나라에서 어떻게 헌법에서 규정되어 있는지를 좀 습득 하셔야 되지 않을까 라고 하는 게 제 개인적인 생각이에요. 우리가 지금 헌법 파악이라고 하는 거 자체를 갖다가 수업하고 있는 내용이 아닌지라, 아 현실적으로 우리가 좀 성적이 좋은 성적을 받아서 여러분들이 원하는 바 빨리 이루는 것이 가장 중요하지 않겠습니까? 그렇다면 대한민국 헌법에 규정되어 있는 내용들이 무엇 인지를 조금 단순 암기,죄송합니다. 이런 말씀을 드려서, 단순 암기를 하셔야 될 부분들도 분명히 있습니다. 당연히 여러분들이 제도적인 측면이기 때문에 우리나라에서 어떤 제도를 지금 어떠한 방법으로 쓰고 있고, 어떠한 일 자, 어떠한 절차 이런 것들을 갖다가 하고 있는지는 이거는 뭐 이해의 대상이 된다 라기 보다는 아무래도 여러분들이 숙지의 대상이 되지 않을까 라고 하는 생각이 들어요. 그래서 이 헌법 파트에 있는 여러분들 기본권이 나와 있고 그리고 국회라고 하는 부분, 대통령과 행정부, 사법부 그리고 헌법재판소, 이런 내용들이 주로 이루어져 있습니다. 그러니까 우리나라에서는 이런 것들을 어떻게 규정하고 있는지를 좀 차근차근 정리하실 필요가 있지 않을까 싶습니다. 그러면 이 헌법 내용 중에서 이제 첫 번째로 우리가 같이 공부한 거에 될 내용은 우리나라 헌법은 구체적으로 어떤 원리 속에서 이루어 지고 있는지 즉,헌법의 기본 원리가 무엇인지에 대해서 말씀을 드려보

In [158]:
textrank = TextRank(text0)
for row in textrank.summarize(3):
    print(row)
    print()
    print('keywords :',textrank.keywords(5))

['']
['', '주제']
['', '주제', '법적 영역 헌법 파트']
['', '주제', '법적 영역 헌법 파트', '헌법 규정']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', 

['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이론', '민족 민족주의 추구', '민족 야죠 통일 염원 역사적 상황 평화 인접국 사랑 경험', '대한민국 민국 헌법 국제 평화 평화주의 대승 추구', '이념 구체화', '헌법 원리 원리다라 다라', '민주 민주주의 추구', '헌법 원리 야겠', '국민주', '대한민국 민국 주권 국민 헌법', '국민주 헌법 원리', '자유 자유민주주의 민주 민주주의', '자유주의 민주 민주주의 결합 원리', '복지 신경', '헌법 원리 복지 복지국가 국가', '복지 복지국가 국가 원리 말로 사회국가', '복지 복지국가 국가 원리', '헌법 이념 기본원리 원리 일치 4번', '국제 평화 평화주의']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이

['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이론', '민족 민족주의 추구', '민족 야죠 통일 염원 역사적 상황 평화 인접국 사랑 경험', '대한민국 민국 헌법 국제 평화 평화주의 대승 추구', '이념 구체화', '헌법 원리 원리다라 다라', '민주 민주주의 추구', '헌법 원리 야겠', '국민주', '대한민국 민국 주권 국민 헌법', '국민주 헌법 원리', '자유 자유민주주의 민주 민주주의', '자유주의 민주 민주주의 결합 원리', '복지 신경', '헌법 원리 복지 복지국가 국가', '복지 복지국가 국가 원리 말로 사회국가', '복지 복지국가 국가 원리', '헌법 이념 기본원리 원리 일치 4번', '국제 평화 평화주의', '국제 국제평화주의 평화 주목', '국제 국제평화주의 평화 기본적 침략 전쟁 부인', '침략 전쟁 부인', '국제 국제평화주의 평화 대한민국 민국 전쟁 부인', '전쟁 침략 부정', '침략 전쟁 수행', '확대 1번', '평화 파괴 침략국 전쟁', '침략 전쟁 부인 지라 침략국 서서 파병', '평화 유지 파병 가능', '', '대한민국 민국 숙원', '평화 평화통일 통일 원리 헌법 포함']
['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌

['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이론', '민족 민족주의 추구', '민족 야죠 통일 염원 역사적 상황 평화 인접국 사랑 경험', '대한민국 민국 헌법 국제 평화 평화주의 대승 추구', '이념 구체화', '헌법 원리 원리다라 다라', '민주 민주주의 추구', '헌법 원리 야겠', '국민주', '대한민국 민국 주권 국민 헌법', '국민주 헌법 원리', '자유 자유민주주의 민주 민주주의', '자유주의 민주 민주주의 결합 원리', '복지 신경', '헌법 원리 복지 복지국가 국가', '복지 복지국가 국가 원리 말로 사회국가', '복지 복지국가 국가 원리', '헌법 이념 기본원리 원리 일치 4번', '국제 평화 평화주의', '국제 국제평화주의 평화 주목', '국제 국제평화주의 평화 기본적 침략 전쟁 부인', '침략 전쟁 부인', '국제 국제평화주의 평화 대한민국 민국 전쟁 부인', '전쟁 침략 부정', '침략 전쟁 수행', '확대 1번', '평화 파괴 침략국 전쟁', '침략 전쟁 부인 지라 침략국 서서 파병', '평화 유지 파병 가능', '', '대한민국 민국 숙원', '평화 평화통일 통일 원리 헌법 포함', '최근 강조 원리', '문화 국가 원리', '문화 국가 원리 문화국가 예전 민족 창달', '최근 한류 영향력 막강 뉴스', '문화 최근 인식 헌법 원리 문화국가 국가 강조', '6가지 헌법 원리 시기', '헌

['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이론', '민족 민족주의 추구', '민족 야죠 통일 염원 역사적 상황 평화 인접국 사랑 경험', '대한민국 민국 헌법 국제 평화 평화주의 대승 추구', '이념 구체화', '헌법 원리 원리다라 다라', '민주 민주주의 추구', '헌법 원리 야겠', '국민주', '대한민국 민국 주권 국민 헌법', '국민주 헌법 원리', '자유 자유민주주의 민주 민주주의', '자유주의 민주 민주주의 결합 원리', '복지 신경', '헌법 원리 복지 복지국가 국가', '복지 복지국가 국가 원리 말로 사회국가', '복지 복지국가 국가 원리', '헌법 이념 기본원리 원리 일치 4번', '국제 평화 평화주의', '국제 국제평화주의 평화 주목', '국제 국제평화주의 평화 기본적 침략 전쟁 부인', '침략 전쟁 부인', '국제 국제평화주의 평화 대한민국 민국 전쟁 부인', '전쟁 침략 부정', '침략 전쟁 수행', '확대 1번', '평화 파괴 침략국 전쟁', '침략 전쟁 부인 지라 침략국 서서 파병', '평화 유지 파병 가능', '', '대한민국 민국 숙원', '평화 평화통일 통일 원리 헌법 포함', '최근 강조 원리', '문화 국가 원리', '문화 국가 원리 문화국가 예전 민족 창달', '최근 한류 영향력 막강 뉴스', '문화 최근 인식 헌법 원리 문화국가 국가 강조', '6가지 헌법 원리 시기', '헌

...

['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이론', '민족 민족주의 추구', '민족 야죠 통일 염원 역사적 상황 평화 인접국 사랑 경험', '대한민국 민국 헌법 국제 평화 평화주의 대승 추구', '이념 구체화', '헌법 원리 원리다라 다라', '민주 민주주의 추구', '헌법 원리 야겠', '국민주', '대한민국 민국 주권 국민 헌법', '국민주 헌법 원리', '자유 자유민주주의 민주 민주주의', '자유주의 민주 민주주의 결합 원리', '복지 신경', '헌법 원리 복지 복지국가 국가', '복지 복지국가 국가 원리 말로 사회국가', '복지 복지국가 국가 원리', '헌법 이념 기본원리 원리 일치 4번', '국제 평화 평화주의', '국제 국제평화주의 평화 주목', '국제 국제평화주의 평화 기본적 침략 전쟁 부인', '침략 전쟁 부인', '국제 국제평화주의 평화 대한민국 민국 전쟁 부인', '전쟁 침략 부정', '침략 전쟁 수행', '확대 1번', '평화 파괴 침략국 전쟁', '침략 전쟁 부인 지라 침략국 서서 파병', '평화 유지 파병 가능', '', '대한민국 민국 숙원', '평화 평화통일 통일 원리 헌법 포함', '최근 강조 원리', '문화 국가 원리', '문화 국가 원리 문화국가 예전 민족 창달', '최근 한류 영향력 막강 뉴스', '문화 최근 인식 헌법 원리 문화국가 국가 강조', '6가지 헌법 원리 시기', '헌

['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이론', '민족 민족주의 추구', '민족 야죠 통일 염원 역사적 상황 평화 인접국 사랑 경험', '대한민국 민국 헌법 국제 평화 평화주의 대승 추구', '이념 구체화', '헌법 원리 원리다라 다라', '민주 민주주의 추구', '헌법 원리 야겠', '국민주', '대한민국 민국 주권 국민 헌법', '국민주 헌법 원리', '자유 자유민주주의 민주 민주주의', '자유주의 민주 민주주의 결합 원리', '복지 신경', '헌법 원리 복지 복지국가 국가', '복지 복지국가 국가 원리 말로 사회국가', '복지 복지국가 국가 원리', '헌법 이념 기본원리 원리 일치 4번', '국제 평화 평화주의', '국제 국제평화주의 평화 주목', '국제 국제평화주의 평화 기본적 침략 전쟁 부인', '침략 전쟁 부인', '국제 국제평화주의 평화 대한민국 민국 전쟁 부인', '전쟁 침략 부정', '침략 전쟁 수행', '확대 1번', '평화 파괴 침략국 전쟁', '침략 전쟁 부인 지라 침략국 서서 파병', '평화 유지 파병 가능', '', '대한민국 민국 숙원', '평화 평화통일 통일 원리 헌법 포함', '최근 강조 원리', '문화 국가 원리', '문화 국가 원리 문화국가 예전 민족 창달', '최근 한류 영향력 막강 뉴스', '문화 최근 인식 헌법 원리 문화국가 국가 강조', '6가지 헌법 원리 시기', '헌

['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이론', '민족 민족주의 추구', '민족 야죠 통일 염원 역사적 상황 평화 인접국 사랑 경험', '대한민국 민국 헌법 국제 평화 평화주의 대승 추구', '이념 구체화', '헌법 원리 원리다라 다라', '민주 민주주의 추구', '헌법 원리 야겠', '국민주', '대한민국 민국 주권 국민 헌법', '국민주 헌법 원리', '자유 자유민주주의 민주 민주주의', '자유주의 민주 민주주의 결합 원리', '복지 신경', '헌법 원리 복지 복지국가 국가', '복지 복지국가 국가 원리 말로 사회국가', '복지 복지국가 국가 원리', '헌법 이념 기본원리 원리 일치 4번', '국제 평화 평화주의', '국제 국제평화주의 평화 주목', '국제 국제평화주의 평화 기본적 침략 전쟁 부인', '침략 전쟁 부인', '국제 국제평화주의 평화 대한민국 민국 전쟁 부인', '전쟁 침략 부정', '침략 전쟁 수행', '확대 1번', '평화 파괴 침략국 전쟁', '침략 전쟁 부인 지라 침략국 서서 파병', '평화 유지 파병 가능', '', '대한민국 민국 숙원', '평화 평화통일 통일 원리 헌법 포함', '최근 강조 원리', '문화 국가 원리', '문화 국가 원리 문화국가 예전 민족 창달', '최근 한류 영향력 막강 뉴스', '문화 최근 인식 헌법 원리 문화국가 국가 강조', '6가지 헌법 원리 시기', '헌

['', '주제', '법적 영역 헌법 파트', '헌법 규정', '헌법 파트', '한정 입장', '노력 필요', '헌법 규정 습득 개인적', '헌법 파악 자체 수업 현실적 실적 성적 중요', '대한민국 민국 헌법 규정 단순 암기', '말씀 단순 암기', '제도적 측면 제도 방법 절차 이거 이해 숙지', '헌법 파트 기본권 국회 행정부 사법부 헌법재판소 재판소', '규정 필요', '헌법 구체적 원리 말씀', '그림 헌법 원리 1번', '헌법 원리 파트', '파티 민주 민주주의 원리 혼동 가능성', '', '헌법 원리 구체적 추구', '추구', '이념 말씀', '헌법 이념', '민주 민주주의', '이론', '민족 민족주의 추구', '민족 야죠 통일 염원 역사적 상황 평화 인접국 사랑 경험', '대한민국 민국 헌법 국제 평화 평화주의 대승 추구', '이념 구체화', '헌법 원리 원리다라 다라', '민주 민주주의 추구', '헌법 원리 야겠', '국민주', '대한민국 민국 주권 국민 헌법', '국민주 헌법 원리', '자유 자유민주주의 민주 민주주의', '자유주의 민주 민주주의 결합 원리', '복지 신경', '헌법 원리 복지 복지국가 국가', '복지 복지국가 국가 원리 말로 사회국가', '복지 복지국가 국가 원리', '헌법 이념 기본원리 원리 일치 4번', '국제 평화 평화주의', '국제 국제평화주의 평화 주목', '국제 국제평화주의 평화 기본적 침략 전쟁 부인', '침략 전쟁 부인', '국제 국제평화주의 평화 대한민국 민국 전쟁 부인', '전쟁 침략 부정', '침략 전쟁 수행', '확대 1번', '평화 파괴 침략국 전쟁', '침략 전쟁 부인 지라 침략국 서서 파병', '평화 유지 파병 가능', '', '대한민국 민국 숙원', '평화 평화통일 통일 원리 헌법 포함', '최근 강조 원리', '문화 국가 원리', '문화 국가 원리 문화국가 예전 민족 창달', '최근 한류 영향력 막강 뉴스', '문화 최근 인식 헌법 원리 문화국가 국가 강조', '6가지 헌법 원리 시기', '헌

{'주제': 338, '법적': 143, '영역': 244, '헌법': 427, '파트': 397, '규정': 64, '한정': 413, '입장': 286, '노력': 78, '필요': 410, '습득': 214, '개인적': 19, '파악': 396, '자체': 297, '수업': 204, '현실적': 431, '실적': 223, '성적': 190, '중요': 342, '대한민국': 100, '민국': 123, '단순': 89, '암기': 232, '말씀': 113, '제도적': 318, '측면': 378, '제도': 317, '방법': 137, '절차': 311, '이거': 270, '이해': 277, '숙지': 210, '기본권': 71, '국회': 55, '행정부': 423, '사법부': 170, '헌법재판소': 429, '재판소': 305, '구체적': 39, '원리': 255, '그림': 65, '1번': 2, '파티': 398, '민주': 128, '민주주의': 129, '혼동': 434, '가능성': 10, '추구': 374, '이념': 273, '이론': 274, '민족': 126, '민족주의': 127, '야죠': 234, '통일': 387, '염원': 243, '역사적': 238, '상황': 180, '평화': 402, '인접국': 280, '사랑': 168, '경험': 27, '국제': 53, '평화주의': 403, '대승': 94, '구체화': 40, '원리다라': 256, '다라': 87, '야겠': 233, '국민주': 49, '주권': 336, '국민': 48, '자유': 291, '자유민주주의': 293, '자유주의': 294, '결합': 26, '복지': 153, '신경': 217, '복지국가': 154, '국가': 41, '말로': 112, '사회국가': 173, '기본원리': 72, '일치': 281, '4번': 5, '국제평화주의': 54, '주목': 337, '기본적': 73, '침략': 379, '전쟁': 308, '부

In [160]:
keyword_cnt = []
for keyword in textrank.keywords(5) :
    keyword_cnt.append(text0.count(keyword))
keyword_cnt

[72, 61, 34, 47, 33]

In [156]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

example = text0
stop_words = bul_list

word_tokens = word_tokenize(example)
# stop_words=stop_words.split(' ')

result = [] 
for w in word_tokens: 
    if w not in stop_words: 
        result.append(w) 

print(word_tokens,'\n') 
print(result)

['반갑습니다.여러분', '.', '한', '주', '동안', '공부', '열심히', '하셨습니까', '?', '이제', '또', '새로운', '주제로', '여러분들을', '공부해야', '될', '시간이', '왔습니다', '.', '아', '이번', '시간은', '법적', '영역에서', '헌법', '파트이지요', '.', '헌법에는', '어떤', '내용들이', '규정되어', '있는지에', '대해서', '여러분들하고', '같이', '공부를', '하게', '됩니다', '.', '그런데', '여러분들이', '헌법', '파트', '라고', '하는', '부분이', '참', '재미있고', '참', '심오합니다', '.', '그런데', '이제', '한정된', '시간', '내에서', '문제를', '풀어야', '되는', '입장이지요', '.', '이럴', '때는', '조금', '여러분들의', '노력이', '여기서부터는', '조금', '더', '많이', '필요하지', '않을까', '라고', '하는', '생각이', '들거든요', '.', '어', '여기', '같은', '경우에는', '일단은', '여러분들', ',', '어', '우리나라에서', '어떻게', '헌법에서', '규정되어', '있는지를', '좀', '습득', '하셔야', '되지', '않을까', '라고', '하는', '게', '제', '개인적인', '생각이에요', '.', '우리가', '지금', '헌법', '파악이라고', '하는', '거', '자체를', '갖다가', '수업하고', '있는', '내용이', '아닌지라', ',', '아', '현실적으로', '우리가', '좀', '성적이', '좋은', '성적을', '받아서', '여러분들이', '원하는', '바', '빨리', '이루는', '것이', '가장', '중요하지', '않겠습니까', '?', '그렇다면', '대한민국', '헌법에', '규정되어', '있는', '내용들이', '무엇', '인지를', '조금', '단순', '암기', ',', '죄송합니다', '.', '이런', '말씀을

In [153]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SMHRD\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [146]:
i = 0
for st in text3.split() :
    if st.find("기본권") != 0 :
        i += 1
i

1933

In [72]:
from collections import Counter
kkma = Kkma()
noun = []
sentence = kkma.sentences(text3)
for st in sentence :
    nouns=kkma.nouns(st)
    for n in nouns : 
        noun.append(n)
count = Counter(noun)

# noun_list = count.most_common(1)
# for v in noun_list:
#     print(v)

In [73]:
count

Counter({'제가': 7,
         '안녕': 1,
         '행정법': 4,
         '장점': 1,
         '오늘': 4,
         '분': 5,
         '샘': 1,
         '기초': 3,
         '입문': 1,
         '특강': 1,
         '거': 34,
         '이번': 1,
         '수업': 3,
         '이': 4,
         '법학': 3,
         '번': 4,
         '공부': 4,
         '대상': 1,
         '데': 4,
         '만약': 5,
         '전체': 2,
         '조망': 1,
         '내용': 6,
         '준비': 1,
         '때문': 4,
         '아빠': 1,
         '5': 2,
         '5만': 1,
         '만': 4,
         '원': 5,
         '정도': 4,
         '수': 12,
         '때': 9,
         '말씀': 3,
         '진짜': 2,
         '필기': 3,
         '마세': 1,
         '이해': 1,
         '전제': 2,
         '시험': 1,
         '일부': 1,
         '우리': 9,
         '이대': 1,
         '배': 2,
         '은': 6,
         '지금': 1,
         '과정': 2,
         '여러분': 6,
         '뭐': 32,
         '옛날': 1,
         '얘기': 5,
         '뜻': 1,
         '저': 6,
         '지정': 3,
         '마음': 3,
         '막': 1,
    

In [70]:
noun

[['제가'],
 ['안녕'],
 ['행정법', '장점'],
 ['오늘', '분', '샘', '기초', '입문', '특강', '행정법', '거'],
 ['이번', '수업', '이', '법학', '번', '공부', '분', '대상'],
 ['데', '만약', '법학', '공부'],
 ['이',
  '전체',
  '조망',
  '기초',
  '내용',
  '제가',
  '준비',
  '때문',
  '아빠',
  '5',
  '5만',
  '만',
  '원',
  '정도'],
 ['분', '수'],
 ['오늘',
  '수업',
  '때',
  '제가',
  '말씀',
  '거',
  '번',
  '진짜',
  '수',
  '필기',
  '마세',
  '내용',
  '법학',
  '이해',
  '전제'],
 ['시험', '내용', '일부', '우리', '행정법', '이대', '배', '은', '전체', '수'],
 ['지금', '기초', '과정', '여러분', '뭐', '옛날', '얘기', '뜻', '저', '필기', '지정', '마음'],
 ['막', '하나하나', '필기', '저', '거', '암기', '스트레스', '내', '전', '내역'],
 ['식', '마음'],
 ['자',
  '우선',
  '민사',
  '민사소송',
  '소송',
  '형사',
  '형사소송',
  '행정',
  '행정소송',
  '개요',
  '게이',
  '법대',
  '수업'],
 ['1',
  '2',
  '학년',
  '때',
  '3',
  '4',
  '배우',
  '심화',
  '심화과정',
  '과정',
  '이것',
  '뭐',
  '우리',
  '뉴스',
  '데',
  '은',
  '내용',
  '거'],
 ['강조',
  '이제',
  '교양',
  '생각',
  '보세',
  '무선',
  '청소기',
  '하면',
  '재판',
  '원고',
  '피고',
  '사이',
  '권리',
  '권리의무',
  '의무',
  '법률',
  '관계',
  '확정',

In [62]:
dics

{'tag': '법률', 'count': 1}

In [13]:
okt1=Okt()

In [14]:
okt1.nouns("뭐냐이거")

['뭐', '거']

In [10]:
import pandas as pd

In [27]:
data = pd.read_csv("./공무원_경제학_구두점_konlpy.csv" , encoding="utf-8-sig")

In [28]:
text = ""
for i in data["new_word"] :
    text += i + " "

In [162]:
textrank = TextRank(text3)

print('keywords :',textrank.keywords())

['제가 안녕']
['제가 안녕', '행정법 장점']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역'

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

In [66]:
data2 = pd.read_csv("./공무원_국어_구두점_konlpy.csv" , encoding="utf-8-sig")

In [ ]:
data

In [67]:
text2 = ""
for i in data2["new_word"] :
    text2 += i + " "

In [68]:
textrank = TextRank(text2)
for row in textrank.summarize(3):
    print(row)
    print()
    print('keywords :',textrank.keywords())

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


우리가 이번 퇴원을 한꺼번에 부를 때 입 퇴원이라 그러지 퇴임 원이라고 하지 않아요.

keywords : ['언어', '형식', '이름', '다라', '문장', '역사', '규칙', '말로', '국어', '영어']
오라고 창가 있는 사람에게 물어 본 것은 뭐냐?

keywords : ['언어', '형식', '이름', '다라', '문장', '역사', '규칙', '말로', '국어', '영어']
그래서 우리가 지금 언어의 기능을 다 살펴봤어요.

keywords : ['언어', '형식', '이름', '다라', '문장', '역사', '규칙', '말로', '국어', '영어']


In [12]:
data3 = pd.read_csv("./공무원_행정법_구두점_konlpy.csv" , encoding="utf-8-sig")

In [13]:
text3 = ""
for i in data3["new_word"] :
    text3 += i + " "

In [23]:
okt = Okt()

In [29]:
kkma = Kkma()

In [31]:
kkma.nouns("청구인용팔결로 인해")

['청구인', '팔결']

['제가 안녕']
['제가 안녕', '행정법 장점']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역'

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '']
['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

['제가 안녕', '행정법 장점', '기초 입문 특강 행정법', '수업 법학', '법학', '전체 조망 기초 제가 준비 아빠', '', '수업 제가 말씀 진짜 필기 마세 법학 이해 전제', '일부 행정법 이대 전체', '기초 옛날 필기 지정 마음', '하나하나 필기 암기 스트레스 내역', '마음', '민사 민사소송 소송 형사 형사소송 행정소송 개요 게이 법대 수업', '학년 배우 심화 심화과정 뉴스', '강조 교양 무선 청소기 재판 원고 피고 사이 권리 권리의무 의무 법률 확정 법원 요구', '여행 여행정보 정보 사법부 법원 요구 재판', '명령', '재판 거구 교정도', '민사 소송법 전제', '제가 민사 민사소송 소송', '민사 민사소송 소송 개인 간의 분쟁 해결 대표적 채무 불이행 이거 아세 채권 만족', '그거 민사 민사소송절차 소송 절차 사인 권리 권리의무 의무 다툼 재판 집회 단순 요쪽', '자동차', '배열', '요가 변제 다라 의리 관리 실현', '이거', '친구 진짜 사업 작하 필요', '나중 이해도 신용 친구 실명 이거 일주일', '약속', '중학교 동창', '', '일주일', '전화', '이거 내가', '', '나중 말씀 상황 입장 소송 지갑 내가 300만 현철', '범죄', '원칙적 사료 구제 허용', '', '사력 구제 국가 가능 현대 법치 법치주의 개인 분쟁 분쟁해결 해결 스스로 멱살 폭행죄 사기죄 고소 협박죄 성립 주거 침입죄 구역 절도 강도 사료', '대신 국가 구제 구제절차 절차 법원 소송 말씀', '제가 법원 다가 소장 제출', '종이 송아지 말로', '청구 취지 원인 양식 소장 제출 민선 나중 불상사 상식적', '청구 취지 판사 판교', '새끼 채권 이행 판결', '하라 권사', '요거 고대 요번 명목 권리 구제 기관', '인도 판사 마음 권리 구제', '청구 취지 고로케 이행 청구권', '가기 이거 청구', '이거 다가 증거', '계약서 가든 요즘 계좌 이체', '계좌 입금 중이 메시지 증거', '통화 청소 보관 의리 불상 

In [107]:
nouns = []
for i in data3["new_word"] :
    for w in kkma.nouns(i) :
        nouns.append(w)

0

['경매', '강의', '차장', '놈', '얼마', '거', '입찰', '사람', '라']

In [41]:
textrank = TextRank(text3)
for row in textrank.summarize(3):
    print(row)
    print()
    print('keywords :',textrank.keywords(30))

그 대신 국가고 잘하는데 국가 구제절차는 어떻게 되냐라고 하면은 법원에 나가 이제 소송을 제기하는 건데 소송이 어떻게 흘러가는지 말씀 드릴게요.

keywords : ['법원', '소송', '민사', '집행', '압류', '판결', '권리', '의무', '인용', '절차', '이거', '청구', '제가', '재판', '나중', '원고', '필요', '피고', '신청', '권리의무', '개인', '해결', '분쟁', '민사소송', '구제', '말씀', '멍멍이', '필기', '계좌', '딱지']
여기 신청 할 때도 역시 소장 비슷하게 제출 하겠네.

keywords : ['법원', '소송', '민사', '집행', '압류', '판결', '권리', '의무', '인용', '절차', '이거', '청구', '제가', '재판', '나중', '원고', '필요', '피고', '신청', '권리의무', '해결', '개인', '분쟁', '민사소송', '구제', '말씀', '멍멍이', '필기', '계좌', '딱지']
신청서를 제출하여 그러면서 그 내 신청 강제집행 해 주세요라는 근거를 같이 갔다 오셨죠.

keywords : ['법원', '소송', '민사', '집행', '압류', '판결', '권리', '의무', '인용', '절차', '이거', '청구', '제가', '재판', '나중', '원고', '필요', '피고', '신청', '권리의무', '해결', '개인', '분쟁', '민사소송', '구제', '말씀', '멍멍이', '필기', '계좌', '딱지']


In [14]:
textrank = TextRank(text3)
for row in textrank.summarize(3):
    print(row)
    print()
    print('keywords :',textrank.keywords())

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


그 대신 국가고 잘하는데 국가 구제절차는 어떻게 되냐라고 하면은 법원에 나가 이제 소송을 제기하는 건데 소송이 어떻게 흘러가는지 말씀 드릴게요.

keywords : ['법원', '소송', '압류', '권리', '청구', '의무', '나중', '재판', '집행', '민사소송']
그래서 여기서 소장 부본을 고대로 송달을 해 줍니다.

keywords : ['법원', '소송', '압류', '권리', '청구', '의무', '나중', '재판', '집행', '민사소송']
그리고 증인도 불러 갖고 물어보기도 하고 이제 결혼 절차를 심리를 한 다음에 그다음에 판결을 내리게 됩니다.

keywords : ['법원', '소송', '압류', '권리', '청구', '의무', '나중', '재판', '집행', '민사소송']


In [13]:
textrank = TextRank(text3)
for row in textrank.summarize(3):
    print(row)
    print()
    print('keywords :',textrank.keywords())

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


그 대신 국가고 잘하는데 국가 구제절차는 어떻게 되냐라고 하면은 법원에 나가 이제 소송을 제기하는 건데 소송이 어떻게 흘러가는지 말씀 드릴게요.

keywords : ['법원', '소송', '압류', '권리', '청구', '의무', '나중', '재판', '집행', '민사소송']
그래서 여기서 소장 부본을 고대로 송달을 해 줍니다.

keywords : ['법원', '소송', '압류', '권리', '청구', '의무', '나중', '재판', '집행', '민사소송']
그리고 증인도 불러 갖고 물어보기도 하고 이제 결혼 절차를 심리를 한 다음에 그다음에 판결을 내리게 됩니다.

keywords : ['법원', '소송', '압류', '권리', '청구', '의무', '나중', '재판', '집행', '민사소송']


In [39]:
data4 = pd.read_csv("./공무원_회계학_구두점_konlpy.csv" , encoding="utf-8-sig")

In [42]:
text4 = ""
for i in data4["new_word"] :
    text4 += i + " "

In [43]:
textrank = TextRank(text4)
for row in textrank.summarize(3):
    print(row)
    print()
    print('keywords :',textrank.keywords())

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


시장에서 자산을 매도할 때 형성되는 가격이에요.

keywords : ['자산', '파트', '손익', '원가', '가치', '포괄', '질문', '수정', '비율', '측정']


keywords : ['자산', '파트', '손익', '원가', '가치', '포괄', '질문', '수정', '비율', '측정']
이쪽 파트에서는 비용이라는 것은 존재하지 않아 수익 일하는 걸 이해하셔야 되는데요 자산이 늘고 그에 따라 자본의 증가하는 것을 수익이라고 부르자 여기는 4가 중이라는 질문자체가 당 기 손익 와 기타 포괄 손익으로 모든 앱 하는 거기 때문에 결과적으로 다산이 얼마가 증가 합니까라는 질문과 동이에요.

keywords : ['자산', '파트', '손익', '원가', '가치', '포괄', '질문', '수정', '비율', '측정']


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
twitter = Twitter()
nouns = []
for sentence in data3["new_word"]:
    if sentence is not '':
        nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
             if noun not in bul_list and len(noun) > 1]))

corpus = nouns

vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(corpus):
    print('====== document[%d] ======' % i)
    print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


====== document[0] ======
[('안녕하십니까', 0.23360203276160776), ('행정법', 0.3820579977487037), ('장점', 0.23360203276160776), ('기초', 0.41739680879126406), ('입문', 0.23360203276160776), ('특강', 0.23360203276160776), ('행정법', 0.3820579977487037), ('수업', 0.19102899887435185), ('법학', 0.41739680879126406), ('보지', 0.23360203276160776), ('법학', 0.41739680879126406), ('전체', 0.20869840439563203), ('기초', 0.41739680879126406), ('보고', 0.20869840439563203), ('준비', 0.23360203276160776), ('아빠', 0.23360203276160776)]
====== document[1] ======
[('분도', 0.3151370848345649), ('수업', 0.257704614606515), ('미리', 0.3151370848345649), ('말씀', 0.257704614606515), ('진짜', 0.257704614606515), ('필기', 0.2815412434273715), ('법학', 0.2815412434273715), ('이해', 0.3151370848345649), ('전제', 0.2815412434273715), ('일부', 0.3151370848345649), ('행정법', 0.257704614606515), ('직접', 0.3151370848345649)]
====== document[2] ======
[('이대용', 0.2693895507464445), ('전체', 0.24067071992910755), ('기초', 0.24067071992910755), ('옛날', 0.2693895507464445), ('필

In [81]:
pip install soyspacing

Note: you may need to restart the kernel to use updated packages.


In [83]:
from soyspacing.countbase import CountSpace

corpus_fname = './data/script/S000068.txt'
model = CountSpace()
model.train(corpus_fname)

all tags length = 41177 --> 1026, (num_doc = 146)

In [84]:
model.save_model(model_fname, json_format=False)

NameError: name 'model_fname' is not defined

In [88]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

# sent = '이건진짜좋은영화 라라랜드진짜좋은영화'
sent = text.replace(" ","")

# with parameters
sent_corrected, tags = model.correct(
    doc=sent,
    verbose=verbose,
    force_abs_threshold=ft,
    nonspace_threshold=nt,
    space_threshold=st,
    min_count=mc)

# without parameters
sent_corrected, tags = model.correct(sent)

print(sent_corrected)

제1장경제기초시작하겠습니다.제일당은독립적으로시험에출제된다기보다는경제공부를함에있어서반드시필요한기초개념과용어들이나와있기때문에이런내용들이있구나!가볍게이해하시고넘어가시면되겠습니다.첫번째논점정지하게기본적개념입니다.만약에우리한테밀가루가한포대있다고.칩시다.밀가루로맛있는빵을만들려고하는데요사람들이빵을좋아하기때문에만약에우리가땅을무한히많이만들수있다.많은사람들이아주행복하게빵을소비할수 있을것입니다. 하지만현실적으로빵을무한히많이만드는것은불가능한데요왜냐하면빵을만드는재료밀가루의양이제한되어있기때문입니다.이처럼자원의부족한자원의희소성으로인해서인간의욕망을충분히충족시키지못하는문제가발생하게되는데우리이러한문제를경제문제라고합니다.이러한경제문제가심각할수록인간은물질적으로충분히행복하지못한문제가발생하게되기때문에이러한경제문제를해결하기위한학문이바로경제학이다 라고정할수가 있습니다.경제하거든요인간의욕망을최대한충족시키기위해서완벽하게충족시킬순없겠죠.왜냐면자원에유한하게때문에하지만최대한충족시키기위해서부족한자원희소한자원을합리적으로가자효율적으로가장경제적으로활용하는방법을연구하는학문을경제학이라고할수가 있겠습니다.즉경제학은경제문제를해결하기위한학문이다주어진자원을최대한합리적효율적경제적으로활용해서각종경제문제를해결하는방법을연구하는모색하는학문분야이다 라고정리할수가 있겠습니다.경제문제는크게세가지문제로대별됩니다.첫번째어떤재활을얼마나생산할것인가우리가밀가루로빵을생산한다없을때빵두종류가다양하게식빵도있고요.소보로빵도있고단팥빵적있습니다.이런식으로주어진밀가루의양은140제한되어 있는데이한포대밀가루로소보로빵은몇개를만들고식빵은몇개를만들고각방에종류마다몇개씩만들것인지생산물의종류수량결정문제가첫번째로있고요.다음에두번째로빵을만들제빵사사람이죠.사람이제빵사no활동에의해석방을생산할수도있고또는공장을자동화시켜서5분이라든가.어떤제빵기계를이용해서빵을생산할수도있습니다.사람의노동력을활용해서빵을생산하는 것을애들이라고쓰고요.기계와같은자본설비를이용해서빵을만드는것을K라고 하죠.이랬을때어떤빵공장에엘과케이의결합비율을결정해야되는데만약에애를많이쓰는경우에는우리가노동집약적으로자를한다고합니다.반대로케이가많

In [118]:
!pip install py-hanspell

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\SMHRD\\AppData\\Local\\Temp\\pip-install-s4juvdze\\py-hanspell_d0f8296a81d94e839a779ca511fe06ef\\setup.py'"'"'; __file__='"'"'C:\\Users\\SMHRD\\AppData\\Local\\Temp\\pip-install-s4juvdze\\py-hanspell_d0f8296a81d94e839a779ca511fe06ef\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\SMHRD\AppData\Local\Temp\pip-pip-egg-info-gzfzf78o'
         cwd: C:\Users\SMHRD\AppData\Local\Temp\pip-install-s4juvdze\py-hanspell_d0f8296a81d94e839a779ca511fe06ef\
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>", line 1

  Using cached py-hanspell-1.1.tar.gz (3.0 kB)
  Using cached py-hanspell-1.0.tar.gz (3.0 kB)


    ----------------------------------------
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\SMHRD\\AppData\\Local\\Temp\\pip-install-s4juvdze\\py-hanspell_e37438e5c9c649c3bf0cef324b6eda59\\setup.py'"'"'; __file__='"'"'C:\\Users\\SMHRD\\AppData\\Local\\Temp\\pip-install-s4juvdze\\py-hanspell_e37438e5c9c649c3bf0cef324b6eda59\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\SMHRD\AppData\Local\Temp\pip-pip-egg-info-s614si2n'
         cwd: C:\Users\SMHRD\AppData\Local\Temp\pip-install-s4juvdze\py-hanspell_e37438e5c9c649c3bf0cef324b6eda59\
    Complete output (5 lines):
    Traceback (most rec

In [142]:
from hanspell import spell_checker

# sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
sent = data["new_word"][41].replace(" ","")
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

일을 한 다이야기입니다. 애를 공급한 다이야기는 회사에 나가서 일을 한 다이야 기고요 공급한 다이야기는요 내 돈을 남한테 빌려준다 이 얘기야 빌려줘서 이자수익을 얻는다 얘기입니다. 아시겠죠. 핸드폰 살 돈을 생산요소 시장에서 과태료 공급해서 돈을 벌고요 2L 밭에는 누가 사나요 바로 기업이 삽니다 기업 EL 과제를 수요해요 그런데 왜 그러면 생산요소가 뭐예요.


In [143]:
 data["new_word"][41]

'일을 한다 이야기입니다. 애를 공급한다 이야기는 회사에 나가서 일을 한다 이야기 고요 공급한다 이야기는요 내 돈을 남한테 빌려 준다이 얘기야 빌려줘서 이자수익을 얻는다 얘기입니다. 아시겠죠. 핸드폰 살 돈을 생산요소시장에서 과태료 공급해서 돈을 벌고 여 2L 밭에는 누가 사나요 바로 기업이 삽 니다 기업 EL 과제를 수요 해요 그런데 왜 그러면 생산요소가 뭐예요. '

In [131]:
 data2["new_word"][1].replace(" ","")

'아가현대국어에서더이상사용되지않고사용되지않고물이현대국어의화서문로형태가바뀌었으며바뀌었으며어디다가어리석다로스있다가현대국어의와서나이가어리다이뜻으로바뀌어쓰이는것등과같은예서알수있는언어의특성은무엇일까요자료는있대.우리가개념어저긴지식이좀없더라도시간의흐름에따라서언어가변화했다.'

In [94]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\smhrd\appdata\local\temp\pip-req-build-aclivz1j


  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\SMHRD\AppData\Local\Temp\pip-req-build-aclivz1j'
  ERROR: Error [WinError 2] 지정된 파일을 찾을 수 없습니다 while executing command git clone -q https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\SMHRD\AppData\Local\Temp\pip-req-build-aclivz1j'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [116]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\smhrd\appdata\local\temp\pip-req-build-g5_jseiz
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\SMHRD\AppData\Local\Temp\pip-req-build-g5_jseiz'
  ERROR: Error [WinError 2] 지정된 파일을 찾을 수 없습니다 while executing command git clone -q https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\SMHRD\AppData\Local\Temp\pip-req-build-g5_jseiz'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [115]:
!pip install git

ERROR: Could not find a version that satisfies the requirement git (from versions: none)
ERROR: No matching distribution found for git


In [15]:
filename='./data/script/S000070.txt'

r = open(filename, mode='rt', encoding='utf-8-sig')
lines = r.readlines()
text0= ""
for line in lines:
    line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
    text0 += line + " "
    print(line)
r.close()

반갑습니다. 여러분 이번 시간 법과 정치 다섯 번째 시간이지요. 이번 시간에는, 어 형법 파트부터 여러분들하고 같이 공부를 하게 될 거에요. 지난 시간에 우리가 민법 영역이라고 하는 거 상당히 자세하게 봤지요.
어 민법 영역만큼 내용은 많지 않지만, 여러분들 형법 영역에서도 중요한 부분들이 몇 가지가 있습니다. 형법이다. 라고 이야기 한다면, 일단 여러분들, 형법의 대원칙이 무엇인지부터 출발을 해야 되겠죠? 바로 죄형법정주의인데 어떤 내용이 있는지 살펴보도록 하겠습니다.
예 여러분들, 형법이라고 하는 것이 있었을 때, 제일 중요한 거 일단 죄형법정주의라고 하는 거에 대한 이해이지요. 죄형법정주의라고 하는 것은 말 그대로 범죄와 형벌을 법으로 정한다. 라고 하는 것으로서, b 대한민국 형법의 대원칙이다. 라고 생각을 하시면 됩니다.
여러분들, 만약에 범죄하고 형벌이라고 하는 것을 법으로 정하지 않는다. 라고 한다면, 공권력의 자의적 해석이라고 하는 것이 가능해질 수가 있겠죠? 그래서 법정 죄형법정주의를 통해서,
어 국민의 기본권이 보호될 수 있다. 라고 하는 거 기억 하셔야 되겠고, 더불어서 기본권에서 신체의 자유를 보호하는 조치 역시 죄형법정주의가 된다. 라고 하는 거 기억이 나시죠?
그렇다면 이 죄형법정주의의 파생원칙에는 무엇이 있을까? 라고 하는 거, 역시 시험 문제의 출제가 될 수가 있는데, 관습형법 금지해야 되고요. 명확해야 되며, 적정성이 유지돼야 됩니다.
그리고 형벌 효력은 불소급의 원칙이지요. 그리고 유추 해석이 금지되는 것, 이 내용 5가지가 죄형법정주의의 내용이 된다고 하는 거 기억을 해두시기 바랍니다.
그럼 여러분들, 범죄는 무엇인가? 라고 이야기 할 때 범죄라고 하는 것은, 당연히 여기 있는 법에 정해져 있어야 되겠고요. 그리고 위법 해야 되겠고요.
그리고 책임성이 있어야 하는데, 때에 따라서는여러분들, 위법해 보이지만 위법성이 깨지는 것들이 있습니다, 그것은 위법성 조각 사유라고 이야기하거든요.
시험 문제 역시 위법성 조각 사유에서 빈출이

In [46]:
textrank = TextRank(text0)
for row in textrank.summarize(3):
    print(row)
    print()
    print('keywords :',textrank.keywords())

라고 해서 성인하고 동일하게 일을 시킬 수는 없겠죠?

keywords : ['국제', '심판', '행정소송', '배심원', '대표', '스스로', '법원', '절차', '동의', '노동']
근로 시간에 있어서 그래서 약간의 제한이 있습니다.

keywords : ['국제', '심판', '행정소송', '배심원', '대표', '스스로', '법원', '절차', '동의', '노동']
라고 하는 것이에요.

keywords : ['국제', '심판', '행정소송', '배심원', '대표', '스스로', '법원', '절차', '동의', '노동']


In [ ]:
import 

In [91]:
textrank = TextRank(text3)
for row in textrank.summarize(20):
    print(row)
    print()
print('keywords :',textrank.keywords(100))

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


있으신 분도 충분히 많은 걸 가져가실 수 있을 겁니다.

민사소송이 다 하고 한번 개인 간의 분쟁을 해결하기 위한 뭐 대표적으로 채무 불이행 이거 뭔지 아세요 우리 돈 안 가는 거 돈 안 갚으면 은 이게 내 채권을 어떻게 만족시켜야 되는가?

사력 구제를 안 하고 국가 구제만 가능하도록 한 게 현대 법치주의 국가요 그래서 개인간 분쟁해결에 생겼을 때 스스로가 가지고 예를 들어서 제가 가지고 야 동 아파 그래서 멱살 잡으면 폭행죄로 어 그런 다음에 뭐 너 안 갚으면 너 사기죄로 고소할 거야 라고 하면 협박죄가 성립되려면 그리고 야 너 돈 갚아 그렇게 밀고 들어가면 주거 침입죄 구역 때리고 그냥 돈 있는 거 갖고 오면 절도 때리고 갖고 오면 볼까요 강도요 사료 구제는 딱 맞고 있어요.

그 대신 국가고 잘하는데 국가 구제절차는 어떻게 되냐라고 하면은 법원에 나가 이제 소송을 제기하는 건데 소송이 어떻게 흘러가는지 말씀 드릴게요.

면 종이에다 쓰는 거 송아지가 가서 말로 하면은 1억을 잊어버릴 수도 있잖아요.

그러면 요거 고대로 할 수 있도록 앞으로도 여러분이 알아두셔야 할 거 뭐냐고 나 요번은 명목상으로는 권리 구제 기관이지만 엔간하면 잘 안 해 줘요.

그런데 바보 한 입장에서는 일도 누 군지 모르고 값도 뭔지 모르겠어 뭐야 그죠.

아무 생각도 안 하고 여기 종이에 써 져 있는 놈한테 그대로 보내

그래서 여기서 소장 부본을 고대로 송달을 해 줍니다.

 아니에요.

 없죠.

그리고 증인도 불러 갖고 물어보기도 하고 이제 결혼 절차를 심리를 한 다음에 그다음에 판결을 내리게 됩니다.

판결을 할 때 원고의 예를 원고를 원고 원고의 청구가 이유가 있다라고 하면 인용판결 내리고 이유가 없다라고 하면 기각 판결 내린 그래서 기각판결 혹은 인용 판결을 내리게 됩니다.

근데 달라진 것은 아무것도 없어요.

이제 시작이야 이제 시작이야 그러면 요렇게 됐다.

여기 신청 할 때도 역시 소장 비슷하게 제출 하겠네.

신청서를 제출하여 그러면서 그 내 신청 강제집행 해

In [34]:
!pip install lexrank

In [48]:
from lexrankr import LexRank

string_all = '''
대한민국은 민주공화국이다.
제1조 대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.
제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.
제2조 ② 국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.
제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.
제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.
제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.
제5조 ② 국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.
제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.
제6조 ② 외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.
제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.
제7조 ② 공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 보장된다.
제8조 ① 정당의 설립은 자유이며, 복수정당제는 보장된다.
제8조 ② 정당은 그 목적·조직과 활동이 민주적이어야 하며, 국민의 정치적 의사형성에 참여하는데 필요한 조직을 가져야 한다.
제8조 ③ 정당은 법률이 정하는 바에 의하여 국가의 보호를 받으며, 국가는 법률이 정하는 바에 의하여 정당운영에 필요한 자금을 보조할 수 있다.
제8조 ④ 정당의 목적이나 활동이 민주적 기본질서에 위배될 때에는 정부는 헌법재판소에 그 해산을 제소할 수 있고, 정당은 헌법재판소의 심판에 의하여 해산된다.
제9조 국가는 전통문화의 계승·발전과 민족문화의 창달에 노력하여야 한다.
'''
 
lexrank = LexRank()  # can init with various settings
lexrank.summarize(string_all)
num_line = 0.3
summaries = lexrank.probe(num_line)  # `num_summaries` can be `None` (using auto-detected topics)
for summary in summaries:
    print(summary)

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


TypeError: __init__() missing 1 required positional argument: 'tokenizer'

In [51]:
from typing import List
from konlpy.tag import Okt

class OktTokenizer:
    okt: Okt = Okt()

    def __call__(self, text: str) -> List[str]:
        tokens: List[str] = self.okt.pos(text, norm=True, stem=True, join=True)
        return tokens

In [53]:
from typing import List
from lexrankr import LexRank

# 1. init
mytokenizer: OktTokenizer = OktTokenizer()
lexrank: LexRank = LexRank(mytokenizer)

# 2. summarize (like, pre-computation)
lexrank.summarize(text3)

# 3. probe (like, query-time)
summaries: List[str] = lexrank.probe(0.05)
for summary in summaries:
    print(summary)

그런데 만약에 법학 공부가 되어 있다고
그리고 오늘 수업 들으실 때 제가 미리 말씀 드리고 싶은 거는 이렇게 여러 번 진짜 수 없듯이 막 필기하고 외우려고 하지 마세요 오늘 내용들은 일단 법학을 이해하기 위한 전제 내용들을 제가 쭉 넓게 다를 거라
하더라도 1 2 학년 때는 안 배우고 3 4 학년 때 배우는 좀 심화과정 있긴 한데 근데 이게 뭐 우리가 많이 접해 보긴 해야 뉴스 같은 데서 많이 나오니까 그러니까 듣다 보면 은 뭐 이렇게 익숙한 내용도 많이 나오실 거예요
그리고 증인도 불러 갖고 물어보기도 하고 이제 결혼 절차를  심리를 한 다음에 그다음에 판결을 내리게 됩니다
지금 그러니까 이 주머니에 1월 꽂히게 하려면 또 뭐가 필요하냐고 이 게 소송절차고 두 번째로 집행절차  민사집행이라고 부르는데 이것도 역시 4번 에다가 청구를 하는데 요 때는 요렇게 삼각 구도가 형성되지만 집행 절차는 신청인 신청인이 법원에 신청을 합니다
말로는 경매라고 하는데 부동산은 실제로는 입찰절차의 하거든요


In [50]:
from typing import List
from lexrankr import LexRank

# 1. init
mytokenizer: OktTokenizer = OktTokenizer()
lexrank: LexRank = LexRank(mytokenizer)

# 2. summarize (like, pre-computation)
lexrank.summarize(text2)

# 3. probe (like, query-time)
summaries: List[str] = lexrank.probe(0.1)
for summary in summaries:
    print(summary)

C:\Users\SMHRD\Anaconda3\envs\Deep_GPU2021\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


NameError: name 'OktTokenizer' is not defined

In [47]:
!pip install lexrankr

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=cc83ecff3b993cce286155115bb694ac212467e1f0d5cee2526e928c66f29e5b
  Stored in directory: c:\users\smhrd\appdata\local\pip\cache\wheels\23\9d\42\5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


In [46]:
from lexrank import STOPWORDS, LexRank
from path import Path

documents = [text0]
# documents_dir = Path('bbc/politics')

# for file_path in documents_dir.files('*.txt'):
#     with file_path.open(mode='rt', encoding='utf-8') as fp:
#         documents.append(fp.readlines())

lxr = LexRank(documents, stopwords=bul_list)

sentences = [
    'One of David Cameron\'s closest friends and Conservative allies, '
    'George Osborne rose rapidly after becoming MP for Tatton in 2001.',

    'Michael Howard promoted him from shadow chief secretary to the '
    'Treasury to shadow chancellor in May 2005, at the age of 34.',

    'Mr Osborne took a key role in the election campaign and has been at '
    'the forefront of the debate on how to deal with the recession and '
    'the UK\'s spending deficit.',

    'Even before Mr Cameron became leader the two were being likened to '
    'Labour\'s Blair/Brown duo. The two have emulated them by becoming '
    'prime minister and chancellor, but will want to avoid the spats.',

    'Before entering Parliament, he was a special adviser in the '
    'agriculture department when the Tories were in government and later '
    'served as political secretary to William Hague.',

    'The BBC understands that as chancellor, Mr Osborne, along with the '
    'Treasury will retain responsibility for overseeing banks and '
    'financial regulation.',

    'Mr Osborne said the coalition government was planning to change the '
    'tax system \"to make it fairer for people on low and middle '
    'incomes\", and undertake \"long-term structural reform\" of the '
    'banking sector, education and the welfare state.',
]

# get summary with classical LexRank algorithm
summary = lxr.get_summary(text3, summary_size=2, threshold=.1)
print(summary)

[' ', '그']


In [19]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from collections import Counter
okt = Okt()

In [29]:
okt.nouns("행정조사를 하기 위해 행정절차를 살폈다")

['행정조사', '위해', '행정절차']

In [49]:

noun=okt.nouns(text3)
for i,v in enumerate(noun) :
    if len(v)<2:
        noun.pop(i)
count = Counter(noun)

noun_list = count.most_common(100)
for v in noun_list :
    print(v)

('이제', 27)
('가지', 20)
('법원', 18)
('뭐', 13)
('판결', 11)
('우리', 10)
('소장', 10)
('부동산', 10)
('그냥', 9)
('거', 9)
('소송', 9)
('돈', 9)
('절차', 9)
('신청', 9)
('제', 8)
('내용', 8)
('다음', 8)
('청구', 8)
('여기', 7)
('제출', 7)
('한번', 6)
('여러분', 6)
('원고', 6)
('결정', 6)
('구제', 6)
('인용', 6)
('집행', 6)
('경매', 6)
('행정법', 5)
('오늘', 5)
('만약', 5)
('얘기', 5)
('민사소송', 5)
('재판', 5)
('권리', 5)
('것', 5)
('내', 5)
('판사', 5)
('사람', 5)
('압류', 5)
('공부', 4)
('때문', 4)
('정도', 4)
('안', 4)
('생각', 4)
('달라', 4)
('이행', 4)
('제기', 4)
('국가', 4)
('다가', 4)
('취지', 4)
('답변서', 4)
('이유', 4)
('기초', 3)
('수업', 3)
('법학', 3)
('요런', 3)
('말씀', 3)
('진짜', 3)
('필기', 3)
('시험', 3)
('지금', 3)
('지정', 3)
('마음', 3)
('식', 3)
('좀', 3)
('은', 3)
('의무', 3)
('채권', 3)
('자동차', 3)
('친구', 3)
('시작', 3)
('필요', 3)
('바로', 3)
('나중', 3)
('혹시', 3)
('아무', 3)
('놈', 3)
('살', 3)
('원', 3)
('면', 3)
('수도', 3)
('고대로', 3)
('하라', 3)
('해', 3)
('얼마', 3)
('증거', 3)
('벌써', 3)
('예', 3)
('역시', 3)
('판단', 3)
('기각', 3)
('조금', 3)
('구도', 3)
('딱지', 3)
('재산', 3)
('처분', 3)
('금지', 3)
('입찰', 3)
('여러', 2)


In [21]:
import os
file_path = "./data/script/"
file_list = os.listdir(file_path)
text= ""
for file_name in file_list :
    r = open(file_path+file_name, mode='rt', encoding='utf-8-sig')
    lines = r.readlines()
    
    for line in lines:
        line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
        text += line + " "
    r.close()

In [22]:
len(text)

1882174

In [23]:
noun=okt.pos(text)

In [24]:
w_list = []
for i,v in enumerate(noun) :
    if len(v)<2:
        noun.pop(i)
count = Counter(noun)

noun_list = count.most_common()
for v in noun_list :
    w_list.append(v)

In [ ]:
textrank = TextRank(text)
for row in textrank.summarize(0):
    print(row)
    print()
    print('keywords :',textrank.keywords())

In [213]:
wrd_list = []
tag_list = []
cnt_list = []
for w in w_list :O
    wrd_list.append(w[0][0])
    tag_list.append(w[0][1])
    cnt_list.append(w[1])

In [214]:
w_df=pd.DataFrame(zip(wrd_list,tag_list,cnt_list),columns=['단어','품사','횟수'])

In [215]:
w_df.to_csv("단어횟수.csv", encoding="utf-8-sig" , index=False) 

In [172]:
text4

''